In [ ]:
import numpy as np
import tensorflow as tf
import sys
import keras
import numpy as np
import tensorflow as tf
from tqdm import tqdm
from tensorflow.keras.layers import Conv2D, MaxPooling2D,BatchNormalization,Conv2DTranspose,Concatenate,Input,Activation,Flatten,Dense
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model
from  tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
# from scipy.io import loadmat
# import h5py
import tensorflow as tf
from tensorflow.keras.utils import register_keras_serializable
import cv2
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
!pip install tqdm
!pip install scikit-learn
!pip install "tensorflow==2.10"
!pip install "numpy<2" --force-reinstall
!pip install matplotlib
!pip install torch
!pip install opencv-python

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
tf.debugging.set_log_device_placement(True)

In [ ]:
dataset = "D:\PythonProjects\TumorPrj\Segementation_1\Segementation_1"

In [ ]:
image_files=sorted(os.listdir('D:\PythonProjects\TumorPrj\Segementation_1\Segementation_1\images'))
mask_files=sorted(os.listdir('D:\PythonProjects\TumorPrj\Segementation_1\Segementation_1\masks'))

In [ ]:
print(image_files[:10])
print(mask_files[:10])

In [ ]:
def display_image_mask(image_path,mask_path):
    count=0
    for image_file,mask_file in zip(image_files,mask_files):
        if count>=6:
            break
        image_path_full=os.path.join(image_path,image_file)
        mask_path_full=os.path.join(mask_path,mask_file)

        image=cv2.imread(image_path_full)
        mask=cv2.imread(mask_path_full,cv2.IMREAD_GRAYSCALE)

        if image is None:
                raise ValueError(f"Image at path {imagepath_full} could not be loaded")
        if mask is None:
                raise ValueError(f"Mask at path {maskpath_full} could not be loaded")

        mask_3ch=cv2.merge([mask,mask,mask])
        image_rgb=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(10,10))

        plt.subplot(1, 3, 1)
        plt.imshow(image)
        plt.title('Image')

        plt.subplot(1, 3, 2)
        plt.imshow(mask)
        plt.title('Mask')

        plt.subplot(1,3,3)
        overlay = cv2.addWeighted(image_rgb, 0.7, mask_3ch, 0.3, 0)
        plt.imshow(overlay)
        plt.title("Image with overlay")

        plt.show()
        count+=1

In [ ]:
image_path='D:\PythonProjects\TumorPrj\Segementation_1\Segementation_1\images'
mask_path='D:\PythonProjects\TumorPrj\Segementation_1\Segementation_1\masks'

In [ ]:
display_image_mask(image_path,mask_path)


In [ ]:
images_size=[]
masks_size=[]
#Check images shape
for image_file, mask_file in tqdm(zip(image_files, mask_files)):

    imagepath_full = os.path.join(image_path, image_file)
    maskpath_full = os.path.join(mask_path, mask_file)

    image = cv2.imread(imagepath_full)
    mask = cv2.imread(maskpath_full)

    images_size.append(image.shape)
    masks_size.append(mask.shape)

In [ ]:
print(list(set(images_size)))
print(list(set(masks_size)))

In [ ]:
SIZE=224
CHANNEL=1

In [ ]:
def preprocessing_images_masks(num_samples=None):
    images=[]
    masks=[]

    if num_samples is not None:
        selected_image_files = image_files[:num_samples]
        selected_mask_files = mask_files[:num_samples]
    else:
        selected_image_files = image_files
        selected_mask_files = mask_files

    for image_file, mask_file in tqdm(zip(selected_image_files, selected_mask_files)):
        imagepathfull=os.path.join(image_path,image_file)
        maskpathfull=os.path.join(mask_path,mask_file)

        image = cv2.imread(imagepathfull,cv2.IMREAD_GRAYSCALE)  #load img
        image = cv2.resize(image,(SIZE,SIZE))
        image=cv2.cvtColor(image,cv2.COLOR_GRAY2RGB)
        image = image/255.0  #normalizes pixel values 0-1
        images.append(image)

        mask = cv2.imread(maskpathfull,cv2.IMREAD_GRAYSCALE)
        mask = cv2.resize(mask,(SIZE,SIZE))
        mask = mask/255.0
        masks.append(mask)
    #add a channel to mask (depth of information per pixel(1))
    return np.array(images), np.expand_dims(np.array(masks), axis=-1)

In [ ]:
X,y=preprocessing_images_masks(num_samples=600)


In [ ]:
print('Shape of X:',X.shape)
print('Shape of y:',y.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [ ]:
print('The shape of X_train',X_train.shape)
print('The shape of y_train',y_train.shape)
print('The shape of X_test',X_test.shape)
print('The shape of y_test',y_test.shape)

In [ ]:
#avoid division by zero
smooth = 1e-15

#decor- save custom function with model
@keras.saving.register_keras_serializable()
#metric for overlap between prediction and ground truth
def dice_coef(y_true, y_pred):
    #convert multi dimensions to 1D
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)

    #common positive pixels
    intersection = tf.reduce_sum(y_true * y_pred)

    #balances TN and FP
    return (2. * intersection + smooth) / (
        tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth
    )

# @register_keras_serializable()
@keras.saving.register_keras_serializable()
#define loss function base one coef
def dice_loss(y_true, y_pred):
    #Reduce tumor mismatch
    return 1.0 - dice_coef(y_true, y_pred)

In [ ]:
# log tensorflow informations
tf.debugging.set_log_device_placement(True)

In [ ]:
#data augmentation
train_datagen=(ImageDataGenerator
               (brightness_range=(0.9,1.1),#Randomly adjust brightness
               zoom_range=[.9,1.1],#Random zoom applied
               fill_mode='nearest') #Prevents unnatural
               )
val_datagen=ImageDataGenerator()

In [ ]:
#extracts features and learn pattern from the input image
def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input) #learn patterns
    x = BatchNormalization()(x)  #stabilizes training
    x = Activation("relu")(x)    #network to learn complex features.(Introduces non-linearity)

    #combine learn pattern into meaningful region
    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

In [ ]:
#increase resolution and merge encoder info in tumor mask
def decoder_block(input, skip_features, num_filters):
    #Upsampling (double spatial resolution)
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

In [ ]:

# def build_resnet50_unet(input_shape):
#     """ Input """
#     inputs = Input(shape=input_shape)
#
#     """ Pre-trained ResNet50 Model """
#     resnet50 = ResNet50(include_top=False, weights="imagenet", input_tensor=inputs)
#
#     """ Encoder """
#     s1 = inputs                                         ## (224 x 224) - Use inputs directly
#     s2 = resnet50.get_layer("conv1_relu").output        ## (112 x 112)
#     s3 = resnet50.get_layer("conv2_block3_out").output  ## (56 x 56)
#     s4 = resnet50.get_layer("conv3_block4_out").output  ## (28 x 28)
#
#     """ Bridge """
#     b1 = resnet50.get_layer("conv4_block6_out").output  ## (14 x 14)
#
#     """ Decoder """
#     d1 = decoder_block(b1, s4, 512)                     ## (28 x 28)
#     d2 = decoder_block(d1, s3, 256)                     ## (56 x 56)
#     d3 = decoder_block(d2, s2, 128)                     ## (112 x 112)
#     d4 = decoder_block(d3, s1, 64)                      ## (224 x 224)
#
#     """ Output """
#     outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)
#
#     model = Model(inputs, outputs, name="ResNet50_U-Net")
#     return model
#
# input_shape = (224, 224, 3)
# model = build_resnet50_unet(input_shape)
# model.summary()

In [ ]:
def build_light_resnet50_unet(input_shape):
    inputs = Input(shape=input_shape)
    resnet50 = ResNet50(include_top=False, weights="imagenet", input_tensor=inputs)

    # Encoder (skip connections)
    s1 = inputs                                      # 224 x 224 x 3
    s2 = resnet50.get_layer("conv1_relu").output     # 112 x 112 x 64
    s3 = resnet50.get_layer("conv2_block3_out").output  # 56 x 56 x 256

    # Bridge(compressed view)
    b1 = resnet50.get_layer("conv3_block4_out").output  # 28 x 28 x 512

    # Decoder
    d1 = decoder_block(b1, s3, 128)   # 56 x 56
    d2 = decoder_block(d1, s2, 64)    # 112 x 112
    d3 = decoder_block(d2, s1, 32)    # 224 x 224

    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d3)

    model = Model(inputs, outputs, name="LightResNet50_U-Net")
    return model


input_shape = (224, 224, 3)
model = build_light_resnet50_unet(input_shape)
model.summary()